# Pipeline 
- Insertar Imagen
- Procesar Imagen
- Separar en líneas
- Enviar a los distintos OCRs
- Correcciones 
- Decisión Final

## Insertar Imagen

# Procesar Imagen

## Separar en líneas

## Enviar a los distintos OCRs

## Correcciones

In [1]:
from typing import List
from serialization.serialization import correction, combine, load_corrector_model
import os
import google.generativeai as genai
from dotenv import load_dotenv

results_from_ocr = [
    [
        "Essta es una prueva de ectraccion de teexto. Connoscida cosa sea a todos los queesta carta uieren como yo don fffffff por la gracia de dios hey de Castiella.",
        "Asta carta fue escrita en el anno del senyor de 1493, y  cosas de gran ymportancia para los reynos."
    ],
    [
        "Essa s una prsueva de etraccion de . Connoda cosa sea a tdos as los qeesta carta ueren como yo don Fferrando por la gracia de d hey de Castiella.",
        "Asta s fue escrita en el ano de seyor de 143, y consigna cossdas de gran ymportana para los reynos."
    ],
]

# Cargar claves de entorno para Gemini
load_dotenv()
GENAI_API_KEY = os.getenv("GENAI_API_KEY")
genai.configure(api_key=GENAI_API_KEY)

spacy_model = "es_core_news_sm"
frequency_dictionary_path = "serialization/spanish_frequency_dictionary.txt"

# Cargar modelo SymSpell y Spacy
nlp, sym_spell = load_corrector_model(
    spacy_model=spacy_model,
    frequency_dictionary_path=frequency_dictionary_path,
    max_edit_distance=3,
    prefix_length=7
)

gemini_model_name="gemini-2.0-flash-exp"

# Crear el sistema de instrucciones para Gemini
chat = genai.GenerativeModel(
    model_name=gemini_model_name,
    system_instruction=[
        """
        Eres un asistente que ayuda a refinar textos. Analiza el siguiente texto original y corregido,
        y reescríbelo en un formato limpio, coherente y gramaticalmente correcto. Los textos son documentos históricos
        extraídos mediante OCR, por lo que pueden contener errores típicos del procesamiento OCR. Los documentos son
        cartas en Cuba de los siglos XV y XVI, manten el español antiguo.
        """
    ]
).start_chat(history=[])


c:\Users\Admin\miniconda3\envs\posprocesamiento\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
results_from_corrections: List[List[str]] = []

for document in results_from_ocr:
    aux: List[str] = []
    for line in document:
        aux.append(correction(line, nlp, sym_spell, chat))
    results_from_corrections.append(aux)
    
print(results_from_corrections)

[['Esta es una prueba de extracción de texto. Conocida cosa sea a todos los que esta carta vieren, como yo, don [nombre ilegible], por la gracia de Dios, [rey/señor] de Castilla.', 'Esta carta fue escrita en el año del Señor de 1493, y trata de cosas de gran importancia para los reinos.'], ['Esta es una prueba de extracción. Conocida cosa sea a todos los que esta carta vieren, como yo, don Ferrando, por la gracia de Dios, [rey/señor] de Castilla.', 'Esta fue escrita en el año del Señor de 1493, y consigna cosas de gran importancia para los reinos.']]


## Decisión Final

In [3]:
result:List[str] = []

# Crear el modelo generativo y configurar el sistema de instrucciones
chat2 = genai.GenerativeModel(
    model_name=gemini_model_name,
    system_instruction="""
    Eres un asistente que ayuda a refinar textos. Analiza las siguientes representaciones de una línea de texto,
    y reescribe solo una en un formato limpio, coherente y gramaticalmente correcto. Los textos son documentos históricos
    extraídos mediante OCR, por lo que pueden contener errores típicos del procesamiento OCR. Los documentos son
    cartas en Cuba de los siglos XV y XVI, manten el español antiguo en tu respuesta.
    """
).start_chat(history=[])

for it in range(len(results_from_ocr[0])):
    aux: List[str] = []
    for i in range(len(results_from_ocr)):
        aux.append(results_from_corrections[i][it])
    
    result.append(combine(aux, chat2))

print("RESULT FINAL")
print(result)

RESULT FINAL
['Conocida cosa sea a todos los que esta carta vieren, como yo, don Ferrando, por la gracia de Dios, [rey/señor] de Castilla.', 'Esta fue escrita en el año del Señor de 1493, y consigna cosas de gran importancia para los reinos.']
